In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-default-prediction-parquet-files/test_file_23.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_72.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_61.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_55.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_71.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_53.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_30.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_20.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_80.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_85.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_36.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_50.pqt
/kaggle/input/amex-default-prediction-parquet-files/test_file_97.pqt
/kaggle/input/amex-default-prediction-parquet-files/train_file_28.pqt
/kaggle/input/amex-default-predic

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 kB 13.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=41c2380a8526743179177a68ed145a5e869ec0fde198de5c0a1e8a6f7f5c6363
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.5
    Uninstalling py4j-0.10.9.5:
      Successfully uninstalled py4j-0.10.9.5


In [3]:
from pyspark.sql import SparkSession
 
spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "15g") \
    .appName('amex-test') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/01 07:08:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
test_df = spark.read.parquet('../input/amex-default-prediction-parquet-files/test*.pqt')

In [5]:
print(f"Total Number of rows: {test_df.count()}, Total Number of columns: {len(test_df.columns)}")

Total Number of rows: 11363762, Total Number of columns: 190


In [6]:
from pyspark.sql.types import StringType, StructType
import pyspark.sql.functions as func 

In [7]:
test_df = test_df.withColumn("customer_ID",test_df["customer_ID"].cast(StringType()))

In [8]:
cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

In [9]:
num_cols = []
for col in test_df.columns[2:]: # exclude the customer id and date columns
    if col not in cat_cols:
        num_cols.append(col)

In [10]:
%%time
count_exprs = {col: 'count' for col in cat_cols}
cat_count_df = test_df.groupby('customer_ID').agg(count_exprs).orderBy('customer_ID', ascending=True).toPandas()

CPU times: user 10 s, sys: 1.14 s, total: 11.1 s
Wall time: 30.3 s


In [11]:
for col in cat_count_df.columns[1:]:
    cat_count_df[col] = cat_count_df[col].astype('int8')

In [12]:
gc.collect()

58

In [13]:
%%time
last_exprs = {col: 'last' for col in cat_cols}
cat_last_df = test_df.groupby('customer_ID').agg(last_exprs).orderBy('customer_ID', ascending=True).toPandas().drop('customer_ID', axis=1)

CPU times: user 8.15 s, sys: 1.07 s, total: 9.22 s
Wall time: 27.8 s


In [14]:
for col in cat_last_df:
    if cat_last_df[col].dtype == "float32":
        cat_last_df[col] = cat_last_df[col].astype('float16')

In [15]:
gc.collect()

51

In [16]:
def grp_unique_count(test_df):
    agg_df = []
    for col in cat_cols:
        agg_df.append(test_df.groupby('customer_ID').agg(func.expr(f'count(distinct {col})').alias(f'nunique({col})')).orderBy('customer_ID', ascending=True).toPandas().drop('customer_ID', axis=1))
    final_df = pd.concat(agg_df, axis=1).astype('int8')
    gc.collect()
    return final_df

In [17]:
%%time
cat_nc_df = grp_unique_count(test_df)

CPU times: user 57.8 s, sys: 616 ms, total: 58.4 s
Wall time: 2min 32s


In [18]:
cat_cols_df = pd.concat([cat_count_df, cat_last_df, cat_nc_df], axis=1)

In [19]:
cat_cols_df.to_pickle('cat_cols_df.pkl', compression='gzip')

In [20]:
del(cat_count_df)
del(cat_last_df)
del(cat_nc_df)
del(cat_cols_df)

In [21]:
gc.collect()

84

In [22]:
def grp_num_cols(test_df):
    def agg_num(agg_func):
        agg_df = []
        for i in range(0, len(num_cols), 20):
            exprs = {col: agg_func for col in num_cols[i:i+20]}
            agg_df.append(test_df.groupBy('customer_ID').agg(exprs).orderBy('customer_ID', ascending=True).toPandas().drop('customer_ID', axis=1))
        final_df = pd.concat(agg_df, axis=1).astype('float16')
        gc.collect()
        return final_df
    num_mean_df = agg_num("mean")
    num_std_df = agg_num("std")
    num_min_df = agg_num("min")
    num_max_df = agg_num("max")
    final_df = pd.concat([num_mean_df, num_std_df, num_min_df, num_max_df], axis=1)
    return final_df

In [23]:
%%time
num_cols_df = grp_num_cols(test_df)

22/06/01 07:13:07 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


CPU times: user 7min 24s, sys: 54.8 s, total: 8min 19s
Wall time: 19min 48s


In [24]:
num_cols_df.to_pickle('num_cols_df.pkl', compression='gzip')

In [25]:
del(num_cols_df)

In [26]:
gc.collect()

63

In [27]:
cat_cols_df = pd.read_pickle('./cat_cols_df.pkl', compression='gzip')

In [28]:
num_cols_df = pd.read_pickle('./num_cols_df.pkl', compression='gzip')

In [29]:
final_test_df = pd.concat([cat_cols_df, num_cols_df], axis=1)

In [30]:
del(cat_cols_df)
del(num_cols_df)

In [31]:
gc.collect()

42

In [32]:
final_test_df.to_pickle('test_agg.pkl', compression='gzip')